# Group 4 Assignment 2 

### Authors: 
-  Chin Yee Wan 
-  Darrel Koh
-  Nguyen Gia Khanh 
-  Ngo Vu Anh	

# Discover and Visualise the Data

## Import Libraries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors

from pyspark.mllib.regression import LabeledPoint

### Read in file

In [ ]:
train_df = pd.read_csv('GA2Datasets/UNSW_NB15_training-set.csv')
test_df = pd.read_csv('GA2Datasets/UNSW_NB15_testing-set.csv')

In [ ]:
spark = SparkSession.builder \
                            .appName("CSCI316GP2")\
                            .config("spark.sql.files.maxPartitionBytes", "1000000")\
                            .getOrCreate()

In [ ]:
spark_df = spark.createDataFrame(train_df)
spark_df.show()

## Functions definition 

#### Custom pipeline for data pre-processing

In [ ]:
class PreProcessPipeline:
    def __init__(self, label_encode = True, process_label = True):
        self.label_encode = label_encode
        self.process_label = process_label
    
    def fit(self):
        return self

    def transform(self, df):
        df = df.drop('id', axis=1)
        df = df.copy()
        if self.label_encode:
            columns = ['proto', 'service', 'state']
            for column in columns:
                unique_values = df[column].unique()
                mapping = {value: index for index, value in enumerate(unique_values)}
                df[column] = df[column].map(mapping)

        if self.process_label:
            def label_transformer(category):
                if category == 'Normal':
                    return 0
                elif category in ['Reconnaissance', 'Analysis', 'Fuzzers', 'Shellcode', 'Generic']:
                    return 0
                elif category in ['Backdoor', 'DoS', 'Exploits', 'Worms']:
                    return 1

            df['label'] = df['attack_cat'].apply(label_transformer)
            df.drop('attack_cat', axis=1, inplace=True)

        return df

#### For visualisation

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### For Evaluation

In [ ]:
def evaluate_model(model, val_data):
    # Make predictions on the validation data
    predictions = model.transform(val_data)

    # Evaluate the model using a BinaryClassificationEvaluator for AUC
    auc_evaluator = BinaryClassificationEvaluator(labelCol='label')
    auc = auc_evaluator.evaluate(predictions)

    # Evaluate the model using a BinaryClassificationEvaluator for areaUnderPR
    pr_evaluator = BinaryClassificationEvaluator(labelCol='label', metricName='areaUnderPR')
    area_under_pr = pr_evaluator.evaluate(predictions)

    # Calculate additional metrics
    sensitivity = predictions.filter("label = 1 and prediction = 1").count() / predictions.filter("label = 1").count()
    specificity = predictions.filter("label = 0 and prediction = 0").count() / predictions.filter("label = 0").count()
    precision = predictions.filter("prediction = 1").count() / predictions.filter("prediction = 1 or prediction = 0").count()
    recall = sensitivity
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (predictions.filter("label = prediction").count()) / predictions.count()
    # youdens_j = sensitivity + specificity - 1
    # balanced_accuracy = (sensitivity + specificity) / 2

    # Create a dictionary of model evaluation metrics
    eval_metrics = {
        'AUC': auc,
        'AreaUnderPR': area_under_pr,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score,
        'Accuracy': accuracy,
        # "Youden's J Index": youdens_j,
        # 'Balanced Accuracy': balanced_accuracy
    }

    return eval_metrics


## Data Exploration

### Convert Spark to Pandas

In [ ]:
# Convert Spark DataFrames to Pandas
# train_df = train_df.toPandas()
# test_df_pandas = test_df.toPandas()
# features_df_pandas = features_df.toPandas()

### Explore the Features set

In [ ]:
# features_df.head(20)
# features_df.show()

### Explore Training Dataset

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.info()
# train_df.printSchema()

In [ ]:
# Calculate the number of zeros in each column:
zero_counts = {}

for column in train_df.columns:
    zero_counts[column] = (train_df[column] == "0").sum()
    
# Create a DataFrame from the zero_counts dictionary
zero_counts_df = pd.DataFrame(list(zero_counts.items()), columns=['Column', 'Zero Count'])

# Display the DataFrame
print(zero_counts_df)


In [ ]:
train_df.describe()

#### There are 4 Objects that will require Encoding
- proto 
- service
- state
- attack_cat (1 of target variables)

##### Proto Attribute:

In [ ]:
# Read the unique values of the column 'proto' in the dataframe 'train_df' 
unique_values = train_df['proto'].unique()
unique_values

In [ ]:
# Group data by 'proto' and calculate the sum for each category
proto_sum = train_df.groupby('proto').size()

# Plotting
plt.figure(figsize=(10, 6))
proto_sum.plot(kind='bar')
plt.title('Sum of Records by Proto')
plt.xlabel('Service')
plt.ylabel('Sum of Records')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
proto_sum

##### Service Attributes:
- Convert '-' to 0
- The rest normal

In [ ]:
# Read the unique values of the column 'service' in the dataframe 'train_df' 
unique_values = train_df['service'].unique()
unique_values

In [ ]:
# Group data by 'service' and calculate the sum for each category
service_sum = train_df.groupby('service').size()

# Plotting
plt.figure(figsize=(10, 6))
service_sum.plot(kind='bar')
plt.title('Sum of Records by Service')
plt.xlabel('Service')
plt.ylabel('Sum of Records')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
service_sum

##### State Attribute:

In [ ]:
# Read the unique values of the column 'state' in the dataframe 'train_df' 
unique_values = train_df['state'].unique()
unique_values

In [ ]:
# Group data by 'state' and calculate the sum for each category
state_sum = train_df.groupby('state').size()

# Plotting
plt.figure(figsize=(10, 6))
state_sum.plot(kind='bar')
plt.title('Sum of Records by State')
plt.xlabel('State')
plt.ylabel('Sum of Records')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
state_sum

##### Attack_cat Attribute:

In [ ]:
# Read the unique values of the column 'attack_cat' in the dataframe 'train_df' 
# Target variable: attack_cat
unique_values = train_df['attack_cat'].unique()
unique_values

In [ ]:
# Group data by 'attack_cat' and calculate the sum for each category
attack_cat_sum = train_df.groupby('attack_cat').size()

# Plotting
plt.figure(figsize=(10, 6))
attack_cat_sum.plot(kind='bar')
plt.title('Sum of Records by Attack Category')
plt.xlabel('Attack Category')
plt.ylabel('Sum of Records')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
attack_cat_sum

In [ ]:
data = {
    'attack_cat': ['Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers', 'Generic', 'Normal', 'Reconnaissance', 'Shellcode', 'Worms'],
    'count': [677, 583, 4089, 11132, 6062, 18871, 37000, 3496, 378, 44]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Calculate the total count of records
total_count = df['count'].sum()

# Calculate the percentage for each attribute
df['percentage'] = (df['count'] / total_count) * 100

print(df)

#### Can consider dropping

- tcprtt,Float,"TCP connection setup round-trip time, the sum of ’synack’ and ’ackdat’." -use this is sufficient, 41k '0' values
- synack,Float,"TCP connection setup time, the time between the SYN and the SYN_ACK packets." - drop
- ackdat,Float,"TCP connection setup time, the time between the SYN_ACK and the ACK packets." -  drop
- ct_ftp_cmd,, No of flows that has a command in ftp session. 81652 '0' values - drop
- ct_flw_http_mthd, No. of flows that has methods such as Get and Post in http service. 74752 '0' values - drop
#### Can consider User-Transformed features
- Involved in the creation of the following features:
    - srcip: Source IP address - not in 
    - dstip: Destination IP address - not in
    - sport: Source port number - not in
    - dsport: Destination port number- not in
    - sttl: Source to destination time to live value 
    - dttl: Destination to source time to live value
    - state: "Indicates to the state and its dependent protocol, e.g. ACC, CLO, CON, ECO, ECR, FIN, INT, MAS, PAR, REQ, RST, TST, TXD, URH, URN, and (-) (if not used state)"
    - service: "http, ftp, smtp, ssh, dns, ftp-data ,irc  and (-) if not much used service"
    - response_body_len: Actual uncompressed content size of the data transferred from the server’s http service
    
##### Attributes not in the dataset:
- is_sm_ips_ports,Binary,"If source (srcip) and destination (dstip) IP addresses equal and port numbers (sport)(dsport)  equal then, this variable takes value 1 else 0"
- ct_state_ttl,Integer,No. for each state (state) according to specific range of values for source/destination time to live (sttl) (dttl).
##### Attributes still in the dataset:
- ct_srv_src,integer,No. of connections that contain the same service (service) and source address (srcip) in 100 connections according to the last time (response_body_len).
- ct_srv_dst,integer,No. of connections that contain the same service (service) and destination address (dstip) in 100 connections according to the last time (response_body_len).
- ct_dst_ltm,integer,No. of connections of the same destination address (dstip) in 100 connections according to the last time (response_body_len).
- ct_src_ ltm,integer,No. of connections of the same source address (srcip) in 100 connections according to the last time (response_body_len).
- ct_src_dport_ltm,integer,No of connections of the same source address (srcip) and the destination port (dsport) in 100 connections according to the last time (response_body_len).
- ct_dst_sport_ltm,integer,No of connections of the same destination address (dstip) and the source port (sport) in 100 connections according to the last time (response_body_len).
- ct_dst_src_ltm,integer,No of connections of the same source (srcip) and the destination (dstip) address in in 100 connections according to the last time (response_body_len).

  


##### tcprtt attribute:

In [ ]:
unique_values = train_df.groupby('tcprtt').size()
unique_values

# train_df['tcprtt']

##### is_sm_ips_ports attribute:

In [ ]:
unique_values = train_df['is_sm_ips_ports'].unique()
unique_values

## Data Visualisation

In [ ]:
rcParams["figure.figsize"]=(20,22)
train_df.hist()
plt.grid()

# Prepare the data

In [ ]:
pipeline = PreProcessPipeline(label_encode=True, process_label=True)
train_df = pipeline.transform(train_df)

### Convert Pandas back to Spark

In [ ]:
# Convert Pandas DataFrames back to Spark DataFrames
# train_df_spark = spark.createDataFrame(train_df)
# test_df_spark = spark.createDataFrame(test_df_pandas)

spark_df = spark.createDataFrame(train_df)
spark_df.show()

print(type(spark_df))

## Assemble columns into a dataframe

In [ ]:
# Repartition the DataFrame into a desired number of partitions
num_partitions = 500
repartitioned_df = spark_df.repartition(num_partitions)

In [ ]:
# Define the input columns (excluding the 'label' column)
input_columns = spark_df.columns[:-1]  # Exclude the last column ('label')

# Create a VectorAssembler to combine the input columns into a single 'features' column
assembler = VectorAssembler(inputCols=input_columns, outputCol='features')


# Transform the DataFrame to add the 'features' column
assembled_df = assembler.transform(repartitioned_df)

## Split Data

In [ ]:
# Split the data into training and validation sets
train_data, val_data = assembled_df.randomSplit([0.8, 0.2], seed=123)

In [ ]:
train_data.select('features').show(truncate=False)

# Model selection and training
- Select machine learning models (Logistic Regression , Decision Tree, Random Forest, Multilayer perceptron).
- Split the data into training and validation sets.
- Train the selected models using the training data.

## Logistic Regression model

### Default Model

In [ ]:
# Example usage for default and fine-tuned models
lr_default = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)
pipeline_default = Pipeline(stages=[lr_default])
model_default = pipeline_default.fit(train_data)

# Evaluate models using the evaluate_model function
evaluation_results_default = evaluate_model(model_default, val_data)

# Print evaluation results for default model
print("Default Model Evaluation:")
for metric, value in evaluation_results_default.items():
    print(f"{metric}: {value}")

In [ ]:
# Example usage for default and fine-tuned models
lr_default = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)
pipeline_default = Pipeline(stages=[lr_default])
model_default = pipeline_default.fit(assembled_df)


# # Build the logistic regression model
# lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)

# # Create a pipeline for the model
# pipeline = Pipeline(stages=[lr])

# # Fit the model on the training data
# model = pipeline.fit(train_data)

# # Make predictions on the validation data
# predictions = model.transform(val_data)

# # Evaluate the model using a BinaryClassificationEvaluator
# evaluator = BinaryClassificationEvaluator(labelCol='label')
# auc = evaluator.evaluate(predictions)

# print("AUC = ", auc)

# print("Default Model Evaluation:")
# for metric, value in evaluation_results_default.items():
#     print(f"{metric}: {value}")


###  Fine-Tuning 

### Evaluation

## Decision Tree Model

### Default Model

###  Fine-Tuning 

### Evaluation

## Random Forest Model

### Default Model

###  Fine-Tuning 

### Evaluation

## Multilayer perceptron Model

### Default Model

###  Fine-Tuning 

### Evaluation

# Final comparison between Models